# References:
- https://www.tensorflow.org/get_started/estimator

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib
import urllib.request

import numpy as np
import tensorflow as tf

In [3]:
#Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [8]:
# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
        f.write(raw)

In [9]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
  filename=IRIS_TRAINING,
  target_dtype=np.int,
  features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
  filename=IRIS_TEST,
  target_dtype=np.int,
  features_dtype=np.float32)

In [10]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=3,
                                      model_dir="/tmp/iris_model")
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": np.array(training_set.data)},
  y=np.array(training_set.target),
  num_epochs=None,
  shuffle=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [11]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 330.486, step = 1
INFO:tensorflow:global_step/sec: 381.368
INFO:tensorflow:loss = 15.0609, step = 101 (0.263 sec)
INFO:tensorflow:global_step/sec: 426.706
INFO:tensorflow:loss = 10.4694, step = 201 (0.234 sec)
INFO:tensorflow:global_step/sec: 337.219
INFO:tensorflow:loss = 5.61057, step = 301 (0.297 sec)
INFO:tensorflow:global_step/sec: 440.424
INFO:tensorflow:loss = 3.70392, step = 401 (0.227 sec)
INFO:tensorflow:global_step/sec: 394.181
INFO:tensorflow:loss = 3.97407, step = 501 (0.254 sec)
INFO:tensorflow:global_step/sec: 172.021
INFO:tensorflow:loss = 11.7063, step = 601 (0.581 sec)
INFO:tensorflow:global_step/sec: 436.09
INFO:tensorflow:loss = 11.8779, step = 701 (0.229 sec)
INFO:tensorflow:global_step/sec: 400.02
INFO:tensorflow:loss = 5.15115, step = 801 (0.250 sec)
INFO:tensorflow:global_step/sec: 431.805
INFO:tensorflow:loss = 11.7943, st

In [12]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": np.array(test_set.data)},
  y=np.array(test_set.target),
  num_epochs=1,
  shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-10-25-05:07:01
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-10-25-05:07:01
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0517145, global_step = 2000, loss = 1.55143

Test Accuracy: 0.966667



In [13]:
# Classify two new flower samples.
new_samples = np.array(
  [[6.4, 3.2, 4.5, 1.5],
   [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": new_samples},
  num_epochs=1,
  shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
  "New Samples, Class Predictions:    {}\n"
  .format(predicted_classes))

INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]

